In [ ]:
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest,chi2 
from sklearn import svm 
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler,RobustScaler
from sklearn.model_selection import train_test_split,KFold,GridSearchCV,cross_val_score,StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, make_scorer,confusion_matrix
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle
from sklearn.ensemble import GradientBoostingClassifier
import os
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.dummy import DummyClassifier
!ls
filenames=[]
for i in os.listdir("Data/"):
    if len(i.strip('.csv'))==7:
        filenames.append("Data/"+i)
data = pd.read_csv(filenames[0])
# count number of 0 and 1 in the dataset
data["categorize"].value_counts()
2694/(2694+11369)
samples=[]
for i in os.listdir("Data/"):
    if len(i.strip(".csv"))==7:
        samples.append(i)
samples
file_sizes_blood = [1121212121,
 2303030303,
 1333333333,
 1303030303,
 1090909090,
 803030303,
 1242424242,
 1469696969,
 1363636363,
 1348484848,
 1227272727,
 1272727272,
 1257575757,
 1015151515,
 1136363636,
 1666666666,
 1363636363,
 1439393939,
 1015151515]
for i in samples:
    print(i+" ")
filenames
len(filenames)
# make a pooled dataset merge with start,end,chr and average categorize
data = pd.read_csv(filenames[0])
for file in filenames[1:15]:
    data = pd.concat([data,pd.read_csv(file)])
data = data.drop(columns=["Unnamed: 0"])
data = data.groupby(["start","end","chr"]).mean().reset_index()
data.to_csv("Data/FinalFile_15_datasets_mean.csv")

# make a pooled dataset merge with start,end,chr and average categorize
data = pd.read_csv(filenames[0])
for file in filenames[1:5]:
    data = pd.concat([data,pd.read_csv(file)])
data = data.drop(columns=["Unnamed: 0"])
data = data.groupby(["start","end","chr"]).mean().reset_index()
data.to_csv("Data/FinalFile_5_datasets_mean.csv")

def take_data_preprocess(filename,col="categorize"):
    x=pd.read_csv(filename)
    y=np.array(x[col])
    x=x.drop(["Unnamed: 0"],axis=1)
    y=np.array(y).reshape(len(y)  ,)
    y_1=np.where(y==0)[0]
    y_2=np.where(y==1)[0]
    index=np.random.choice(y_1,len(y_2), replace=False)
    index=np.concatenate((index, y_2), axis=0)
    x_32=x[[ 'AA_x', 'AT_x', 'AG_x', 'AC_x',
       'TA_x', 'TT_x', 'TG_x', 'TC_x', 'GA_x', 'GT_x', 'GG_x', 'GC_x', 'CA_x',
       'CT_x', 'CG_x', 'CC_x', 'AA_y', 'AT_y', 'AG_y', 'AC_y', 'TA_y', 'TT_y',
       'TG_y', 'TC_y', 'GA_y', 'GT_y', 'GG_y', 'GC_y', 'CA_y', 'CT_y', 'CG_y',
       'CC_y']]
    x_32_norm=np.array(normalize(np.array(x_32)))
    x, y = shuffle(x_32_norm[index], y[index], random_state=0)
    return x,y
def classification_report_with_accuracy_score(y_true, y_pred):
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
    return accuracy_score(y_true, y_pred) # return accuracy score
def classification_report_with_f1_score(y_true, y_pred):
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
    return f1_score(y_true, y_pred,average='weighted') # return accuracy score
def confidence_score(y_true,y_pred_prob):

    keys=[0,1]
    dict={}
    i=0
    for key in keys:
        dict[key]=i
        i+=1

    cs=0.0
    for i in range(len(y_true)):
        cs+=y_pred_prob[i][dict[y_true[i]]]


    return cs/len(y_true)
def normalize(x):
    new_x=[]
    
    for i in range(len(x)):
        x_new=[]
        for j in range(16,32):
            a=np.sum(x[i][1:17])
            b=np.sum(x[i][17:33])
            x_new.append(x[i][j]*a/(0.01+ x[i][j-16]*b))
        new_x.append(x_new)
    return new_x
accuracy_blood=[]
for filename in filenames:
    x,y=take_data_preprocess(filename)
    x=StandardScaler().fit_transform(x)
    
    clf = RandomForestClassifier(class_weight='balanced', max_depth=20 ,n_estimators=800, criterion='entropy')
    
    originalclass = []
    predictedclass = []

    outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

    nested_score = cross_val_score(clf, X=x, y=y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))
    
    accuracy_blood.append(sk.metrics.accuracy_score(originalclass, predictedclass))
    
    print(classification_report(originalclass, predictedclass))
accuracy_blood
np.mean(accuracy_blood),np.max(accuracy_blood),np.min(accuracy_blood)
filenames=[]
samples=[]
for i in os.listdir("Data/"):
    if len(i.strip(".csv"))==10 and i!="SRR8670684":
        filenames.append("Data/"+i)
        data = pd.read_csv("Data/"+i)
        samples.append(data.columns[4])
filenames, samples
accuracy_blood_pooled=[]
filenames_pooled = ["Data/FinalFile_5_datasets_mean.csv", "Data/FinalFile_10_datasets_mean.csv", "Data/FinalFile_15_datasets_mean.csv", "Data/FinalFile_20_datasets_mean.csv"]
for filename in filenames_pooled:
    x,y=take_data_preprocess(filename)
    x=StandardScaler().fit_transform(x)
    
    clf = RandomForestClassifier(class_weight='balanced', max_depth=20 ,n_estimators=800, criterion='entropy')
    
    originalclass = []
    predictedclass = []

    outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

    nested_score = cross_val_score(clf, X=x, y=y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))
    
    accuracy_blood_pooled.append(sk.metrics.accuracy_score(originalclass, predictedclass))
    
    print(classification_report(originalclass, predictedclass))
filename
ceph_files = os.listdir("/project/ReadStatistics-data/CEPH_BAM/")[:50]
# open each file directory and get the FinalFile.csv
data = pd.read_csv("/project/ReadStatistics-data/CEPH_BAM/"+ceph_files[0]+"/FinalFile.csv")
print(data.columns)
for file in ceph_files[1:]:
    filename = "/project/ReadStatistics-data/CEPH_BAM/"+file+"/FinalFile.csv"
    if os.path.exists(filename):
        data = pd.concat([data,pd.read_csv(filename)])
data
import os

data = data.groupby(["start","end","chr"]).mean().reset_index()
data.to_csv("Data/FinalFile_50_ceph_datasets_mean.csv")

accuracy_ceph=[]
filename = "Data/FinalFile_50_ceph_datasets_mean.csv"
x,y=take_data_preprocess(filename)
x=StandardScaler().fit_transform(x)

clf = RandomForestClassifier(class_weight='balanced', max_depth=20 ,n_estimators=800, criterion='entropy')

originalclass = []
predictedclass = []

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

nested_score = cross_val_score(clf, X=x, y=y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

accuracy_ceph.append(sk.metrics.accuracy_score(originalclass, predictedclass))

print(classification_report(originalclass, predictedclass))

accuracy_blood_pooled_race=[]
filenames_pooled = ["Data/FinalFile_5_datasets_mean.csv", "Data/FinalFile_10_datasets_mean.csv", "Data/FinalFile_15_datasets_mean.csv", "Data/FinalFile_20_datasets_mean.csv"]
for filename in filenames_pooled:
    x,y=take_data_preprocess(filename)
    x=StandardScaler().fit_transform(x)
    
    clf = RandomForestClassifier(class_weight='balanced', max_depth=20 ,n_estimators=800, criterion='entropy')
    
    originalclass = []
    predictedclass = []

    outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

    nested_score = cross_val_score(clf, X=x, y=y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))
    
    accuracy_blood_pooled.append(sk.metrics.accuracy_score(originalclass, predictedclass))
    
    print(classification_report(originalclass, predictedclass))
np.savez("Figures/accuracy_blood_pooled_data.npz",accuracy_blood_pooled)
cancer_types = samples 
file_sizes=[1303041736,
1205710255,
1696808001,
1183872482,
1259117801,
1301672928,
1177541886,
1229410952,           
979656179]
def plot_roc_curves(model,train_x,train_y,test_x,test_y):
    
    model_dummy = DummyClassifier(strategy='stratified')
    model_dummy.fit(train_x, train_y)
    yhat = model_dummy.predict_proba(test_x)
    naive_probs = yhat[:, 1]


    yhat = model.predict_proba(test_x)
    pos_probs = yhat[:, 1]
    roc_auc = roc_auc_score(test_y, pos_probs)
    fpr, tpr, thresholds = sk.metrics.roc_curve(test_y, pos_probs)
    
    plt.figure(figsize=(7,4))
    plt.plot(fpr, tpr,label="ROC curve of class 1.0 (area = "+str(roc_auc)+")")
    plt.plot([0,1],[0,1],label="",color="black", linestyle='dashed')
    plt.legend()
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC curves")
    plt.show()
def binary(y):
    y_out=[]
    for i in y:
        if i<0.5:
            y_out.append(0.0)
        else:
            y_out.append(1.0)
    return y_out
def take_data_preprocess(filename,col):
    x=pd.read_csv(filename)
    x=x[x[col]!=-1.0]
    y=np.array(x[col])
    y=binary(y)
    x=x.drop(["Unnamed: 0"],axis=1)
    y=np.array(y).reshape(len(y)  ,)
    y_1=np.where(y==0.0)[0]
    y_2=np.where(y==1.0)[0]
    index=np.random.choice(y_1,len(y_2), replace=False)
    index=np.concatenate((index, y_2), axis=0)
    x_32=x[[ 'AA_x', 'AT_x', 'AG_x', 'AC_x',
       'TA_x', 'TT_x', 'TG_x', 'TC_x', 'GA_x', 'GT_x', 'GG_x', 'GC_x', 'CA_x',
       'CT_x', 'CG_x', 'CC_x', 'AA_y', 'AT_y', 'AG_y', 'AC_y', 'TA_y', 'TT_y',
       'TG_y', 'TC_y', 'GA_y', 'GT_y', 'GG_y', 'GC_y', 'CA_y', 'CT_y', 'CG_y',
       'CC_y']]
    x_32_norm=np.array(normalize(np.array(x_32)))
    x, y = shuffle(x_32_norm[index], y[index], random_state=0)
    return x,y
results_cancer=[]
j=0
for filename in filenames:
    
    x,y=take_data_preprocess(filename,cancer_types[j])
    print(filename,len(x))
    x=StandardScaler().fit_transform(x)
    
    
    originalclass = []
    predictedclass = []
    predictedprob = []
    featureimportances = []

    outer_cv = StratifiedKFold(n_splits=10, shuffle=False)

    nested_score = cross_val_score(clf, X=x, y=y, cv=outer_cv, scoring= make_scorer(classification_report_with_f1_score))
    
    for train, test in outer_cv.split(x,y):
            clf = RandomForestClassifier(class_weight='balanced', max_depth=20 ,n_estimators=800, criterion='entropy')
            clf.fit(x[train], y[train])
            
            originalclass.extend(y[test])
            predictedclass.extend(clf.predict(x[test]))
            predictedprob.extend(clf.predict_proba(x[test]))
            featureimportances.append(clf.feature_importances_)
            
            
    
    results_cancer.append([originalclass,predictedclass,predictedprob,featureimportances])
    
    j+=1
    
    print(classification_report(originalclass, predictedclass))
human_data=pd.read_csv("/project/ReadStatistics/MethylationAnalysis/Data/FinalFile_20_datasets_mean.csv")
def get_results(filename,col="categorize"):
    results=[]

    x,y=take_data_preprocess(filename,col)
    x=StandardScaler().fit_transform(x)


    originalclass = []
    predictedclass = []
    predictedprob = []
    featureimportances = []

    outer_cv = StratifiedKFold(n_splits=10, shuffle=False)

    for train, test in outer_cv.split(x,y):
            clf = RandomForestClassifier(class_weight='balanced', max_depth=20 , n_estimators=600) 
            clf.fit(x[train], y[train])

            originalclass.extend(y[test])
            predictedclass.extend(clf.predict(x[test]))
            predictedprob.extend(clf.predict_proba(x[test]))
            featureimportances.append(clf.feature_importances_)



    results.append([originalclass,predictedclass,predictedprob,featureimportances])
    return results
filename_human20 = "/project/ReadStatistics/MethylationAnalysis/Data/FinalFile_20_datasets_mean.csv"
filename_human10 = "/project/ReadStatistics/MethylationAnalysis/Data/FinalFile_10_datasets_mean.csv"
results_human20=get_results(filename_human20)
results_human10=get_results(filename_human10)
samples,accuracy_blood
len(file_sizes_blood)
plt.figure(figsize=(30,10))

# j=0
# for i in results_cancer:
#     acc=accuracy_score(i[0],i[1])
#     plt.scatter(file_sizes[j],acc,label=cancer_types[j],s=200)
#     j+=1

j=0
for i in accuracy_blood[:len(samples)]:
    plt.scatter(file_sizes_blood[j],i,label=samples[j],s=300)
    j+=1


acc=accuracy_score(results_human20[0],results_human20[1])
plt.scatter(816687234*20,acc,label="HUMAN BLOOD 20 SAMPLES",s=300,color="black")

acc=accuracy_score(results_human10[0],results_human10[1])
plt.scatter(816687234*10,acc,label="HUMAN BLOOD 10 SAMPLES",s=300,color="black")
    
plt.xlabel("Number of reads",fontsize=20)
plt.ylabel("Test accuracy",fontsize=20)
plt.xscale("log")
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(title="Sample name",fontsize=15,title_fontsize=15, loc='lower right')

plt.savefig("Figures/test_acc_vs_no_reads.pdf",dpi=1000)

plt.show()
samples
accuracy_blood = np.array(accuracy_blood)
file_sizes_blood = np.array(file_sizes_blood)
samples = np.array(samples)

results_human20 = np.array([results_human20[0][0],results_human20[0][1]])
results_human10 = np.array([results_human10[0][0],results_human10[0][1]])
np.savez("Figures/test_acc_vs_no_reads_data.npz", 
         accuracy_blood=np.array(accuracy_blood, dtype=object),  
         file_sizes_blood=np.array(file_sizes_blood, dtype=object),  
         samples=np.array(samples, dtype=object),  
         acc_human20=results_human20,  
         acc_human10=results_human10)
features=[]
for i in "ATGC":
    for j in "ATGC":
        features.append(i+j)

features_sort=[]
for i in "ACGT":
    for j in "ACGT":
        features_sort.append(i+j)
    
index_sorted=[]
for i in range(16):
    for j in range(16):
        if features[i]==features_sort[j]:
            index_sorted.append(j) 


# Train a model on the human blood data 20 samples and get the feature importances
filename_human20 = "/project/ReadStatistics/MethylationAnalysis/Data/FinalFile_20_datasets_mean.csv"

x,y=take_data_preprocess(filename_human20,"categorize")
x=StandardScaler().fit_transform(x)

clf = RandomForestClassifier(class_weight='balanced', max_depth=20 ,n_estimators=800, criterion='entropy')
clf.fit(x,y)

feature_importances = clf.feature_importances_
plt.figure(figsize=(10,6))
plt.bar(np.arange(0,16),feature_importances[index_sorted],color="blue",label="Feature importances")
plt.xticks(np.arange(0,16)+5*0.1,np.array(features)[index_sorted],fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel("\nFeatures",fontsize=20)
plt.ylabel("\nFeature importances",fontsize=20)
plt.legend(bbox_to_anchor=(1.0, 1.0),title="Sample names",fontsize=15,title_fontsize=15)

plt.tight_layout()


plt.savefig("Figures/feature_importances.pdf",dpi=800)

plt.show()
features
import numpy as np
import matplotlib.pyplot as plt

# Sort feature importances and corresponding feature names
sorted_idx = np.argsort(feature_importances)[::-1]  # Sort in descending order
sorted_importances = feature_importances[sorted_idx]
sorted_features = np.array(features)[sorted_idx]

# Plot
plt.figure(figsize=(12, 6))  # Increase width for better visibility
plt.bar(np.arange(len(sorted_features)), sorted_importances, color="royalblue", alpha=0.8)

# Labels
plt.xticks(np.arange(len(sorted_features)), sorted_features, fontsize=16, rotation=45, ha="right")
plt.yticks(fontsize=16)
plt.xlabel("Features", fontsize=18)
plt.ylabel("Feature Importance", fontsize=18)
plt.title("Feature Importances", fontsize=20, fontweight="bold")

# Save plot
plt.tight_layout()
plt.savefig("Figures/feature_importances.pdf", dpi=800, bbox_inches="tight")

# Show plot
plt.show()

# ✅ Save necessary data for future use
np.savez("Figures/feature_importances_data.npz",
         feature_importances=sorted_importances,
         features=sorted_features)
plt.figure(figsize=(10,6))
j=0
for i in results:
    plt.bar( np.arange(0,32,2)+j*0.1,np.mean(i[3],axis=0)[index_sorted],label=cancer_types[j], width = 0.05)
    j+=1
    
plt.bar( np.arange(0,32,2)+j*0.1,np.mean(results_human20[0][3],axis=0)[index_sorted],label="HUMAN BLOOD 20 SAMPLES", width = 0.05)

plt.xticks(np.arange(0,32,2)+5*0.1,np.array(features)[index_sorted],fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel("\nFeatures",fontsize=20)
plt.ylabel("\nFeature importances",fontsize=20)
plt.legend(bbox_to_anchor=(1.0, 1.0),title="Sample names",fontsize=15,title_fontsize=15)

plt.tight_layout()


plt.savefig("Figures/feature_importances.pdf",dpi=800)

plt.show()

# Train a model on the human blood data 20 samples and test on the cancer data with thresholding 

x,y=take_data_preprocess(filename_human20,"categorize")
x=StandardScaler().fit_transform(x)

clf = RandomForestClassifier(class_weight='balanced', max_depth=20 ,n_estimators=800, criterion='entropy')
clf.fit(x,y)

accuracy_cancer=[]
j=0

for filename in filenames:
    
    
    x,y=take_data_preprocess(filename,cancer_types[j])
    x=StandardScaler().fit_transform(x)
    
    predictedclass = clf.predict(x) 
    
    originalclass = binary(y)
    
    accuracy_cancer.append(sk.metrics.accuracy_score(originalclass, predictedclass))
    
    print(classification_report(originalclass, predictedclass))
    
    j+=1
accuracy_cancer
filenames=[]
for i in os.listdir("Data/"):
    if len(i.strip('.csv'))==7:
        filenames.append("Data/"+i)
filenames

np.mean(accuracy_cancer),np.std(accuracy_cancer),np.mean(accuracy_blood),np.std(accuracy_blood)
# load all the data and make plots 
